## Stokes Sinker Benchmark

### Analytical Solution
The benchmark describes the terminal velocity of a dense sphere sinking in a viscous medium under gravity. Using balance of forces, we can derive the terminal velocity of the sphere as:

\begin{align}
 6 \pi \eta r v_t = \frac{4}{3} \pi r^3 (\rho_s - \rho_f) g,
\end{align}
where:
- $v_t$ is the terminal velocity of the sphere,
- $\eta$ is the dynamic viscosity of the fluid,
- $r $ is the radius of the sphere,
- $\rho_s$ is the density of the sphere,
- $\rho_f$ is the density of the fluid,
- $g$ is the acceleration due to gravity.

Rearranging the equation to solve for terminal velocity $v_t$:
$$ v_t = \frac{2 r^2 (\rho_s - \rho_f) g}{9 \eta} $$

### Numerical Solution
We can simulate this benchmark in ASPECT using the accompanying input file `stokes.prm` (available in [ASPECT cookbooks](https://aspect-documentation.readthedocs.io/en/latest/user/cookbooks/cookbooks/stokes/doc/stokes.html)). The key parameters to set in the input file are:

* `set Density differential for compositional field 1 = 100` ($\rho_s - \rho_f = 100 \, \text{kg/m}^3$)
* `set Viscosity = 1e22` ($\eta = 1 \times 10^{22} \, \text{Pa.s}$)

and the Initial compositional model defines the sphere of radius, $r$, at a point, $p$ using the `Function expression`:

```
subsection Initial composition model
  set Model name = function

  subsection Function
    set Variable names      = x,y,z
    set Function constants  = r=200000,p=1445000
    set Function expression = if(sqrt((x-p)*(x-p)+(y-p)*(y-p)+(z-p)*(z-p)) > r, 0, 1)
  end
end
```

For the given parameters, the analytical terminal velocity is, $v_t = 2*200000^2*100*9.81/(9*1e22) = \mathbf{8.72e-11 m/s}$.

In [ ]:
! aspect ../prm_files/stokes.prm

/bin/bash: line 1: aspect: command not found


In [ ]:
# plot the mesh with pyvista
import pyvista as pv
from glob import glob

In [ ]:
solution_file_names = sorted(glob('output-stokes/solution/solution-*.pvtu'))

plotter = pv.Plotter()
plotter = pv.Plotter(off_screen=False)
mesh = pv.read(solution_file_names[0])

# We will only plot the 2D slice through the model for visualization
y_center = 0.5 * (mesh.bounds[2] + mesh.bounds[3])
slice_mesh = mesh.slice(normal='y', origin=(0, y_center, 0))

# The compositional field represents the blob here
plotter.add_mesh(slice_mesh, scalars='C_1', show_scalar_bar=False)

# Add velocity vectors
glyphs = slice_mesh.glyph(orient='velocity', scale=False, factor=1e5)
plotter.add_mesh(glyphs, color='white', lighting=False)

plotter.add_axes()

# We know that the model is XY plane so the following
# lines just sets up the camera
plotter.view_xz()
plotter.camera.roll = 0
plotter.camera.parallel_projection = True

plotter.show()

Widget(value='<iframe src="http://localhost:46417/index.html?ui=P_0x7c4363e6b150_35&reconnect=auto" class="pyv…

In [ ]:
# Now, extract the velocity field in the sinking sphere represented by the
# compositional field C_1
sinker_composition  = 0.5
compositional_field = mesh['C_1'] > sinker_composition
velocity_sinker     = mesh['velocity'][compositional_field]
avg_sinker_velocity = np.linalg.norm(velocity_sinker.mean(axis=0))

print(f'Average sinking velocity of the sphere: {avg_sinker_velocity:.3e} m/s')

We can see that the numerically computed averaged terminal velocity of the sinker, $\mathbf{8.503e-10 m/s}$ matches closely with the analytical solution, $\mathbf{8.72e-11 m/s}$.

### Exercises

* Change the radius of the sinker to verify that the terminal velocity scales with the square of the radius.
* Make the fluid denser than the sinker to observe buoyant rise instead of sinking. Is the terminal velocity still the same magnitude but opposite direction?
* Vary the viscosity of the fluid to observe its effects on the terminal velocity. Does it inversely scale as expected?